In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); device

In [ ]:
device = "cpu"

In [ ]:
ontology_and_seq__fp = "../data/intermediary/drosophila_protein_ontology_and_seqs.csv"
df = pd.read_csv(ontology_and_seq__fp)
relevant_subset = df[df.qualifier.isin(["enables", "involved_in"])].dropna()
interesting_go_names = [name for (name, freq) in relevant_subset.go_name.value_counts().to_dict().items() if 1 < freq]  # <- probably need to change the filter step !!
df = df[df.go_name.isin(interesting_go_names)]
one_row_per_gene = pd.DataFrame(index=df.seq.unique(), columns=interesting_go_names).fillna(0)
for _, row in df.iterrows():
    one_row_per_gene.loc[row.seq, row.go_name] = 1
one_row_per_gene = one_row_per_gene.reset_index().rename(columns={"index": "seq"})
one_row_per_gene.head()

In [ ]:
df_original = pd.read_csv(ontology_and_seq__fp)
relevant_subset = df_original[df_original.qualifier.isin(["enables", "involved_in"])].dropna()

interesting_go_names = [
    name for (name, freq)
    in relevant_subset.go_name.value_counts().to_dict().items()
    if 5 < freq  # !! probably need to change the filter step !!
]  
relevant_subset = relevant_subset[relevant_subset.go_name.isin(interesting_go_names)]

df = pd.DataFrame(index=relevant_subset.seq.unique(), columns=interesting_go_names).fillna(0)
for _, row in relevant_subset.iterrows():
    df.loc[row.seq, row.go_name] = 1
df["training"] = df.assign(training=0).training.apply(lambda _: random.random() < 0.75)
df = df.reset_index().rename(columns={"index": "seqs"})

vocab = set()
for seq in df.seqs:
    vocab.update(seq)
vocab.add("<pad>")
residue2idx = {char: i for i, char in enumerate(vocab)}

n_examples, n_ontological_categories = df.shape
n_examples, n_ontological_categories

In [ ]:
class SeqOntologyDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.X = list(df.seqs)
        self.y = df.loc[:, interesting_go_names]
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        label = self.y.iloc[i,:].values
        seq = self.X[i]
        seq_tensor = torch.zeros(len(seq), len(vocab))
        for ii, residue in enumerate(seq):
            seq_tensor[ii][residue2idx[residue]] = 1
            # delete this!!! (after pipeline ready -- just to force one category per row)
            break
        return seq_tensor, label

def collate(batch):
    xs, ys = list(zip(*batch))
    xs_padded = pad_sequence(xs, padding_value=residue2idx["<pad>"], batch_first=True)
    return xs_padded, [len(x) for x in xs], torch.tensor(ys)
    
batch_size = 2
dl_test = DataLoader(SeqOntologyDataset(df[~df.training]),
                     batch_size, collate_fn=collate, shuffle=True)
dl_train = DataLoader(SeqOntologyDataset(df[df.training]),
                      batch_size, collate_fn=collate, shuffle=True)
dataloaders = {"train": dl_train, "val": dl_test}

In [ ]:
def train_model(model, n_embedding_dims, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1) + "\n" + '-' * 10)
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            for seq_padded, seqs_len, ontology in dataloaders[phase]:
                seq_padded, ontology = seq_padded.to(device), ontology.to(device)
                batch_size, *_ = seq_padded.shape
                hidden = model.init_hidden(batch_size).to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    activations, hidden_units = model(seq_padded, seqs_len, hidden)
                    _, ontology_pred = torch.max(outputs, 1)
                    loss = criterion(ontology_pred, ontology)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            if phase == "train":
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    return model

n_hidden_units = 16
n_embedding_dims = 16

rnn = nn.RNN(len(vocab), hidden_size=n_hidden_units)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
train_model(rnn, n_embedding_dims, criterion, optimizer, scheduler, num_epochs=25)

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.seq2embedding = nn.Embedding(input_size, embedding_size)
        self.seq2hidden = nn.Linear(embedding_size + hidden_size, hidden_size)
        self.hidden2class = nn.Linear(embedding_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
            
    def forward(self, X, x_lens, hidden):
        X = pack_padded_sequence(X, x_lens, batch_first=True, enforce_sorted=False)
        X = self.seq2embedding(X)
        combined = torch.cat((X, hidden), -1)
        hidden = self.seq2hidden(combined)
        X = self.hidden2class(hidden)
        X = self.softmax(X)  # not apropriate for multiclassification!!
        return X, H

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1) + "\n" + '-' * 10)
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            for seq_padded, seqs_len, ontology in dataloaders[phase]:
                seq_padded, ontology = seq_padded.to(device), ontology.to(device)
                batch_size, *_ = seq_padded.shape
                hidden = model.init_hidden(batch_size).to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    activations, hidden_units = model(seq_padded, seqs_len, hidden)
                    _, ontology_pred = torch.max(outputs, 1)
                    loss = criterion(ontology_pred, ontology)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            if phase == "train":
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    return model

rnn = RNN(len(vocab), 16, 128, len(interesting_go_names))
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
train_model(rnn, criterion, optimizer, scheduler, num_epochs=25)